In [158]:
import os
import warnings
warnings.simplefilter(action = 'ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
def ignore_warn(*args, **kwargs):
    pass

warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="ticks", color_codes=True, font_scale=1.5)
color = sns.color_palette()
sns.set_style('darkgrid')
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
import pylab 
from sklearn import metrics
from scipy import stats
from scipy.stats import skew, norm, probplot, boxcox
from scipy.special import boxcox1p
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import RobustScaler, PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.feature_selection import f_regression, mutual_info_regression, SelectKBest, RFECV, SelectFromModel
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.feature_extraction import FeatureHasher
from sklearn.decomposition import PCA, KernelPCA
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression, OrthogonalMatchingPursuit, Lasso, LassoLarsIC, ElasticNet, ElasticNetCV
from sklearn.linear_model import SGDRegressor, PassiveAggressiveRegressor, HuberRegressor, BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
import lightgbm as lgb
from sklearn import linear_model
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, cross_val_predict, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [141]:
tablin = pd.read_csv(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\Modelo evolución precios\Datos - copia - Marta\tabla_modelos.csv', 
                   sep=';', engine="python", decimal=',', encoding="latin1")
#predict = regresión lineal x imptotal
#predict_act = regresión lineal x imp total actualizado

In [142]:
tablin= tablin.drop(columns=['matricula','latitud','longitud','Actualizacion','Index',
'Provincia', 'PIB per cápita_log', 'Hipotecas','Transacciones inmobiliarias', 'Parque_viviendas_log', 'Euribor', 'Tasa de dudosidad_log','Tasa desempleo',
'Año','Year','parque_cat', 'tasa_cat','valor promedio m2 fomento','Valor metro cuadrado_log','linear_regression','ridge_regression','Codigo_Postal', 
'IMPTOTAL actualizado','Precio m2 actualizado','Precio m2', 'Valor metro cuadrado',
        'CONTAMINACION_TERRENO', 'CONTAMINACION_CONSTRUCCION', 'CONTAMINACION_OTROS', 'EDIF_LICENCIA_COD', 'EDIF_USO_POLIVALENTE',
       'CONTAMINACION_ACUSTICA','CONTAMINACION_AMBIENTAL','VIV_DISTR_NUM_ASCENSORES', 'VIV_INST_TERRAZA_TENDEDERO',
         'Transacciones_inmobiliarias_log','Tasa de dudosidad', 'VIV_INST_AGUA_CALIENTE', 'SUPADAPT', 'SUBTIPO_COD',
        'CONTAMINACION_TERRENO', 'CONTAMINACION_CONSTRUCCION', 'CONTAMINACION_OTROS', 'EDIF_VIV_REGI_LEGAL_DESC',
        'EDIF_LICENCIA_COD','EDIF_USO_POLIVALENTE','VIV_INST_COMU_TENIS_PADEL', 'UBICACION_INMUEBLE_COD_ENC',
                            'UBICACION_INMUEBLE_COD_DEB','Establecimientos_OSM','Parque viviendas', 'PIB per cápita', 'Euribor_log',
                            'CARAC_SINGULARES','VIV_INST_COMU_OTRAS_DEPORT','Fecha_num', 'UBICACION_ZONA_COD_DEB'])
# a partir de la indentación, son variables que no aportaban nada al modelo y las debería quitar a posteriori
#es lo mismo usar los valores de linear_regression que todas las variables

In [ ]:
#Crear variables dummies 
hi= tablin.loc[:,['UBICACION_MUNICIPIO_DESC',
'UBICACION_ZONA_COD','UBICACION_INMUEBLE_COD','Municipio']]
dummy = pd.get_dummies(hi)
tablin = pd.concat([tablin, dummy], axis=1)
tablin = tablin.drop(['UBICACION_MUNICIPIO_DESC',
'UBICACION_ZONA_COD','UBICACION_INMUEBLE_COD','Municipio'], axis=1)

In [143]:
# categorizar viv_superficie terraza
#tablin['VIV_SUPERF_TERRAZA']= [1 if x>0 else x for x in (tablin['VIV_SUPERF_TERRAZA'])]

In [144]:
tablin=pd.DataFrame(tablin)

## Crear variable de variación del precio por año 

In [145]:
#Ordenar por fecha (quarterly)
#tablin= tablin.sort_values('Fecha_num')

#Agrupar por fecha y calcular la media del precio de las viviendas
#tablin['Variacion'] = tablin.groupby('Fecha_num')['IMPTOTAL'].transform('mean')

In [146]:
#Calcular la variacion de la media del precio de las viviendas respecto al trimestre anterior
#c= tablin.loc[:,['Fecha_num', 'Variacion']]
#c= c.drop_duplicates()
#c['change'] = c['Variacion'].pct_change()
#c=c.fillna(0)
#tablin= tablin.merge(c.drop(['Variacion'],1), how='left', on=['Fecha_num'])
#tablin= tablin.drop(columns='Variacion')

In [147]:
#plt.figure(figsize=(30,30))
#sns.heatmap(correlation.corr(),
#           vmin=-1,
#            cmap='coolwarm',
#           annot=True);

In [148]:
#Variables más relacionadas con IMPTOTAL
#tablin = tablin.loc[:,['IMPTOTAL','VIV_DISTR_NUM_ASEOS','SUPERF_ADOPTADA','Valor metro cuadrado','linear_regression']]

In [149]:
#LINEAR REGRESSION PARA GUARDAR PREDICCIONES DE TODAS LAS VIVIENDAS (SIN TRAIN Y TEST)


linear_regressor = linear_model.LinearRegression()

x = tablin.drop(columns=['IMPTOTAL', 'MATRICULA', 'Fecha'])
cols = [*x]
y = tablin.loc[:,'IMPTOTAL']
x = np.asarray(x)
y = np.asarray(y).reshape(len(y), 1)

x_pred = x

xt = x[:,-1].reshape( len(x[:,-1]), 1 )
xt_pred = x_pred[:,-1].reshape( len(x_pred[:,-1]), 1 )

linear_regressor.fit(x, y)

predictions = linear_regressor.predict(x_pred)
score= linear_regressor.score(x, y)

list_predictions = [item for sublist in predictions for item in sublist]
tablin['Predicciones'] = list_predictions

print("R-squared:", score)

R-squared: 0.7705464349089921


In [150]:
round(tablin.loc[:,['IMPTOTAL','Predicciones']].apply(lambda s: abs(s[0]-s[1])/s[0], axis=1).mean(),2)

0.26

In [151]:
tablin.to_csv(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\Modelo evolución precios\Datos - copia - Marta\tabla_linear_regression.csv',
                   sep=';', decimal=',', encoding="latin1", index=False)

In [152]:
tablin= tablin.drop(columns='Predicciones')

In [153]:
#LINEAR REGRESSION PARA EVALUAR TRAIN Y TEST

linear_regressor2 = linear_model.LinearRegression()
train, test = train_test_split(tablin, test_size = 0.33, random_state = 0)

data_with_dummies_train = train
data_with_dummies_test = test

x_train_df = data_with_dummies_train.drop(columns=['IMPTOTAL', 'MATRICULA', 'Fecha'])
y_train_df = data_with_dummies_train.loc[:,'IMPTOTAL']
x_train_array = np.asarray(x_train_df)
y_train_array = np.asarray(y_train_df).reshape( len(y_train_df), 1 )

x_test_df = data_with_dummies_test.drop(columns=['IMPTOTAL', 'MATRICULA', 'Fecha'])
y_test_df = data_with_dummies_test.loc[:,'IMPTOTAL']
x_test_array = np.asarray(x_test_df)
y_test_array = np.asarray(y_test_df).reshape( len(y_test_df), 1 )

linear_regressor2.fit( x_train_array, y_train_array )
    
predict = linear_regressor2.predict(x_test_df)
    
list_predictions = [item for sublist in predict for item in sublist]

data_with_dummies_test['Predict'] = list_predictions

score= linear_regressor2.score(x_train_array, y_train_array)
print("R-squared:", score)

#TABLA FINAL CON TODOS LOS VALORES ACTUALIZADOS

R-squared: 0.7761289375280557


In [154]:
round(data_with_dummies_test.loc[:,['Predict','IMPTOTAL']].apply(lambda s: abs(s[0]-s[1])/s[0], axis=1).mean(),2)

0.29

In [ ]:
#Linear regression assumptions

#Linear relationship
#Multivariate normality
#No or little multicollinearity
#No auto-correlation
#Homoscedasticity 

#regression analysis requires at least 20 cases per independent variable in the analysis

In [47]:
import statsmodels.api as sm

X = pd.DataFrame(tablin, columns=[*tablin])
X.drop(columns=['IMPTOTAL','MATRICULA', 'Fecha'], axis=1, inplace=True)
y = pd.Series(tablin['IMPTOTAL'], name='MEDV')

# inspect data
X.head()

,CARAC_SINGULARES,EDIF_FECHA_CONSTRUCCION,EDIF_CALIDAD_CONSTRU_COD,EDIF_EST_CONSERVACION_COD,EDIF_PLANTA_SOBRE_RASANTE,VIV_ORIENTACION_COD,VIV_SUPERF_TERRAZA,VIV_DISTR_NUM_DORMITORIOS,VIV_DISTR_NUM_ASEOS,VIV_INST_COCINA_EQUIPADA,...,Municipio_Xàtiva,Municipio_Yecla,Municipio_Zamora,Municipio_Zaragoza,Municipio_Águilas,Municipio_Ávila,Municipio_Écija,Municipio_Úbeda,Establecimientos_OSM,change
0,0,206,3.000,0.000,7.000,2,0.000,3,2,1,...,0,0,0,0,0,0,0,0,15,0.000
1,0,158,3.000,4.000,1.000,2,0.000,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000
2,0,160,3.000,2.000,2.000,1,36.000,5,2,1,...,0,0,0,0,0,0,0,0,29,0.000
3,0,150,4.000,0.000,5.000,2,0.000,3,1,0,...,0,0,0,0,0,0,0,0,21,0.000
4,0,160,3.000,6.000,1.000,2,0.000,0,1,0,...,0,0,0,0,0,0,0,0,0,0.000


In [48]:
X_constant = sm.add_constant(X)
lin_reg = sm.OLS(y,X_constant.astype(float)).fit()
lin_reg.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     503.0
Date:                Tue, 06 Aug 2019   Prob (F-statistic):               0.00
Time:                        09:35:36   Log-Likelihood:            -5.1893e+05
No. Observations:               41978   AIC:                         1.038e+06
Df Residuals:                   41693   BIC:                         1.041e+06
Df Model:                         284                                         
Covariance Type:            nonrobust                                         
=======================================================================================================================
                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
const                                               -2.149e+04   1.88e+04     -1.144      0.253   -5.83e+04    1.53e+04
CARAC_SINGULARES                                     1.245e+04   2628.229      4.736      0.000    7295.680    1.76e+04
EDIF_FECHA_CONSTRUCCION                               297.9981     15.644     19.048      0.000     267.335     328.661
EDIF_CALIDAD_CONSTRU_COD                            -2.083e+04    710.170    -29.328      0.000   -2.22e+04   -1.94e+04
EDIF_EST_CONSERVACION_COD                            -409.8167    156.076     -2.626      0.009    -715.728    -103.905
EDIF_PLANTA_SOBRE_RASANTE                             455.8025     99.399      4.586      0.000     260.978     650.627
VIV_ORIENTACION_COD                                 -1.223e+04    874.564    -13.989      0.000   -1.39e+04   -1.05e+04
VIV_SUPERF_TERRAZA                                    164.6217     20.019      8.223      0.000     125.383     203.860
VIV_DISTR_NUM_DORMITORIOS                           -3566.7683    319.688    -11.157      0.000   -4193.363   -2940.174
VIV_DISTR_NUM_ASEOS                                  1.858e+04    512.022     36.282      0.000    1.76e+04    1.96e+04
VIV_INST_COCINA_EQUIPADA                             3753.8631    803.046      4.675      0.000    2179.875    5327.851
VIV_INST_ARMARIOS_EMPOTR                             4771.5131    690.481      6.910      0.000    3418.156    6124.871
VIV_INST_PUERTA_SEGUR                                6253.6300    687.412      9.097      0.000    4906.288    7600.972
VIV_INST_SISTEMA_ALARMA                              2.908e+04   1203.728     24.156      0.000    2.67e+04    3.14e+04
VIV_INST_CIRCUITO_SEGUR                              5344.1366   1227.557      4.353      0.000    2938.099    7750.175
VIV_INST_AIRE_ACOND                                  3068.7413   1094.175      2.805      0.005     924.135    5213.347
VIV_INST_COMU_PISCINA                                2.529e+04    846.153     29.886      0.000    2.36e+04    2.69e+04
VIV_INST_COMU_OTRAS_DEPORT                           3489.8862   1375.372      2.537      0.011     794.128    6185.644
SUPERF_ADOPTADA                                      1105.5605      6.168    179.239      0.000    1093.471    1117.650
Hipotecas_log                                        1.925e+04   2181.426      8.824      0.000     1.5e+04    2.35e+04
Tasa desempleo_log                                  -3.341e+04   2923.760    -11.426      0.000   -3.91e+04   -2.77e+04
Fecha_num                                             175.7266     84.038      2.091      0.037      11.011     340.443
UBICACION_MUNICIPIO_DESC_Nivel medio                -9428.4374   1428.001     -6.60

In [11]:
#Jarque–Bera test is a goodness-of-fit test of whether sample data have the skewness and kurtosis matching a normal distribution.
#If it is far from zero, it signals the data do not have a normal distribution.

In [12]:
#Some notes on the Durbin-Watson test:
#the test statistic always has value between 0 and 4
#value of 2 means that there is no autocorrelation in the sample
#values < 2 indicate positive autocorrelation, values > 2 negative one.

In [13]:
#Expectation (mean) of residuals is zero

#Check for residuals: must be around 0

lin_reg.resid.mean()

1.743589697183925e-09

In [14]:
#TEST OF NORMALITY (DEPENDENT VARIABLE)
from scipy.stats import mstats

#The array needs to be larger than 20, just an example
z,pval = mstats.normaltest(y)

if(pval < 0.05):
    print ("Not normal distribution")
    print (pval)

Not normal distribution
0.0
